<a href="https://colab.research.google.com/github/naman065/Machine-Learning/blob/main/CV_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Task**
Run the code cell by cell!

In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# *Download the DataSet from GitHub and Upload it on your Google Drive*
PS: Dont change any name while uploading otherwise the program will not work!!!

In [3]:

from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/dataset/train'
validation_dir = '/content/drive/MyDrive/dataset/validation'
test_dir = '/content/drive/MyDrive/dataset/test'

Mounted at /content/drive


**DATA AUGMENTATION**

In [4]:

# Data augmentation and normalization for training
# I have done this to randomize the images and avoid overfitting.
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only normalization for validation and test
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 288 images belonging to 2 classes.
Found 99 images belonging to 2 classes.
Found 81 images belonging to 2 classes.


**Pre-Trained Model is used and Custom Layers are added for classification into male and female**

In [5]:
#MobileNetV2 is used for this model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)  #Also Done to Avoid OVERFITTING and simplify the MODEL!!!
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 [==============================] - 0s 0us/step


**Initial Layers of MobileNetV2 is freezed i.e not used and Model is Trained on the corresponding features obtained from pre trained model**

In [6]:
#Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#I used adam optimizer from a similar code of age detection on kaggle!
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
9/9 [==============================] - 80s 9s/step - loss: 1.5902 - accuracy: 0.5243 - val_loss: 0.5404 - val_accuracy: 0.7576
Epoch 2/10
9/9 [==============================] - 20s 2s/step - loss: 0.5643 - accuracy: 0.6979 - val_loss: 0.7810 - val_accuracy: 0.5152
Epoch 3/10
9/9 [==============================] - 18s 2s/step - loss: 0.5114 - accuracy: 0.7326 - val_loss: 0.4706 - val_accuracy: 0.7677
Epoch 4/10
9/9 [==============================] - 18s 2s/step - loss: 0.4506 - accuracy: 0.7708 - val_loss: 0.6159 - val_accuracy: 0.6364
Epoch 5/10
9/9 [==============================] - 18s 2s/step - loss: 0.4027 - accuracy: 0.8299 - val_loss: 0.6462 - val_accuracy: 0.6566
Epoch 6/10
9/9 [==============================] - 20s 2s/step - loss: 0.3837 - accuracy: 0.8368 - val_loss: 0.4321 - val_accuracy: 0.7980
Epoch 7/10
9/9 [==============================] - 18s 2s/step - loss: 0.3379 - accuracy: 0.8368 - val_loss: 0.5784 - val_accuracy: 0.6869
Epoch 8/10
9/9 [==================

***THIS WAS WRITTEN TO IMPROVE UPON ACCURACY DONT RUN AS TAKES TIME P:***

In [ ]:
"""
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)"""


"\nfor layer in base_model.layers[-20:]:\n    layer.trainable = True\n\nmodel.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])\n\nmodel.fit(\n    train_generator,\n    epochs=10,\n    validation_data=validation_generator\n)"

**DataSet also includes a test file to test the accuracy of the model**

In [7]:

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

3/3 [==============================] - 15s 7s/step - loss: 0.3887 - accuracy: 0.8519
Test Accuracy: 0.85
Test Loss: 0.39


# **Upload the image from your computer provided on GitHub**

In [8]:

from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded_image = files.upload()
img_path = list(uploaded_image.keys())[0]

# Image is pre processed here according to model
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Prediction (male or female)
prediction = model.predict(img_array)
print("Prediction:", "Male" if prediction[0][0] > 0.5 else "Female")


Saving myimage.jpg to myimage.jpg
1/1 [==============================] - 2s 2s/step
Prediction: Male


**SAVING THE MODEL**

In [10]:
import shutil
from google.colab import drive
model.save('gender_model.keras')
print("Model saved as gender_model.keras")

# Step 2: Mount Google Drive
drive.mount('/content/drive')

# Step 3: Copy the saved model to Google Drive
shutil.copy('gender_model.keras', '/content/drive/My Drive/gender_model.keras')


Model saved as gender_model.keras
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/gender_model.keras'

# **GRADIO** **INTERFACE**

In [11]:
!pip install gradio
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


model = tf.keras.models.load_model('gender_model.h5')

def predict_gender(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    return "Male" if prediction[0][0] > 0.5 else "Female"


interface = gr.Interface(
    fn=predict_gender,
    inputs=gr.Image(type="pil"),
    outputs="text"
)


interface.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.2 MB/s e